In [68]:
import operator
import copy

operator_dict = {
    "<" : operator.lt,
    "=" : operator.eq,
    ">" : operator.gt,
}

op_list = ["<" , "=" , ">"]

type_dict = {
    "int" : int,
    "str" : str,
    "string" : str,
    "float" : float,
}

class Predicate:
    #Predicates contain an attribute (here a position so a number)
    #and an operator. See operator_dict for all possible operator.
    def __init__(self, attributePos, operator, attributeType):
        self.attribute = attributePos #int
        self.operator = operator #list of string
        self.attributeType = attributeType
    #return true if (row1,row2) does satisfy the predicates
    def is_true(self,row1,row2):
        for op in self.operator:
            if operator_dict[op](type_dict[self.attributeType](row1[self.attribute]),type_dict[self.attributeType](row2[self.attribute])):
                return True
        return False
    def __str__(self):
        return "(" +str(self.__dict__) + ")"

    def __eq__(self, other): 
        return self.__dict__ == other.__dict__
    
    def get_inverse_op(self):
        op2 = []
        for elem in op_list:
            if elem not in self.operator:
                op2.append(elem)
        return op2
    def inverse_predicate(self):
        self.op = self.get_inverse_op()
        
    def partiesliste(self,seq):
        p = []
        i, imax = 0, 2**len(seq)-1
        while i <= imax:
            s = []
            j, jmax = 0, len(seq)-1
            while j <= jmax:
                if (i>>j)&1 == 1:
                    s.append(seq[j])
                j += 1
            p.append(s)
            i += 1 
        return p    
        
    def get_weaker(self):
        weaker_pred = []
        weak_op = self.partiesliste(self.operator)
        for elem in weak_op:
            weaker_pred.append(Predicate(self.attribute,elem,self.attributeType))
        return weaker_pred
    
    def get_weaker_not_empty(self):
        weak = self.get_weaker()
        returnList = []
        for i in range(len(weak)):
            if not(len(weak[i].operator) == 0):
                returnList.append(weak[i])
        return returnList
    def get_weaker_not_empty_different(self):
        weaker = self.get_weaker_not_empty()
        returnList = []
        for elem in weaker:
            if(elem != self):
                returnList.append(elem)
        return returnList
    
    def size(self):
        return len(self.operator)
    
    def var_cost(self, pred, alpha):
        return alpha*(self.size() - pred.size())

Test Predicates class:

In [69]:
data = [['Ayres', '8-8-1984', '322-573', '2007', '21', '0'],['Ayres', '5-1-1960', '***-389', '2007', '22', '0']]

pred1 = Predicate(4,[">"],"int")
print(pred1.is_true(data[0],data[1])==False)
print(pred1.is_true(data[1],data[0])==True)
pred2 = Predicate(5,["<"],"int")
print(pred2.is_true(data[0],data[1])==False)
print(pred2.is_true(data[1],data[0])==False)

pred3 = Predicate(5,["<","="],"int")
for elem in pred3.get_weaker():
    print(elem)
print("___")
for elem in pred3.get_weaker_not_empty():
    print(elem)

print(pred3.var_cost(pred2,2)==2)

pred4=Predicate(5,["<","=",">"],"int")
x=pred4.get_weaker_not_empty()
for elem in x:
    print(elem)
print("___")
y = pred4.get_weaker_not_empty_different()
for elem in y:
    print(elem)

True
True
True
True
({'attribute': 5, 'operator': [], 'attributeType': 'int'})
({'attribute': 5, 'operator': ['<'], 'attributeType': 'int'})
({'attribute': 5, 'operator': ['='], 'attributeType': 'int'})
({'attribute': 5, 'operator': ['<', '='], 'attributeType': 'int'})
___
({'attribute': 5, 'operator': ['<'], 'attributeType': 'int'})
({'attribute': 5, 'operator': ['='], 'attributeType': 'int'})
({'attribute': 5, 'operator': ['<', '='], 'attributeType': 'int'})
True
({'attribute': 5, 'operator': ['<'], 'attributeType': 'int'})
({'attribute': 5, 'operator': ['='], 'attributeType': 'int'})
({'attribute': 5, 'operator': ['<', '='], 'attributeType': 'int'})
({'attribute': 5, 'operator': ['>'], 'attributeType': 'int'})
({'attribute': 5, 'operator': ['<', '>'], 'attributeType': 'int'})
({'attribute': 5, 'operator': ['=', '>'], 'attributeType': 'int'})
({'attribute': 5, 'operator': ['<', '=', '>'], 'attributeType': 'int'})
___
({'attribute': 5, 'operator': ['<'], 'attributeType': 'int'})
({'at

In [214]:
class DC:
     #A denial constraint is a conjunction of multiple predicates.
    #They cannot be all true at the same time
    
    #predicates should be none or a list of predicates!
    def __init__(self,relSize,colType):
        self.predicates = []
        for i in range(relSize):
            self.add(Predicate(i,["<","=",">"],colType[i]))
    
    def add(self,pred):
        self.predicates.append(pred)
        
    def change_predicate(self,predic):
        for pred in self.predicates:
            if pred.attribute == predic.attribute:
                pred.operator = predic.operator
                return None;
        
    def __str__(self):
        stringreturn = "{"
        for pred in self.predicates:
            stringreturn = stringreturn + str(pred) + ","
        return stringreturn + "}"
    
    def size(self):
        return len(self.predicates)
    
    #return true if at least one of the predicates is false    
    def is_satisfy(self,row1,row2):
        for predicate in self.predicates:
            if not(predicate.is_true(row1,row2)):
                return True
        return False
    
    def get_violations(self,data):
        viol_set =[]
        for i in range(len(data)):
            for j in range (len(data)):
                if (i != j and not(self.is_satisfy(data[i],data[j]))):
                    tupleviol = (i,j)         
                    viol_set.append(tupleviol)            
        return viol_set
    
    def is_satisfy_by_data(self,data):
        #data non vide
        if data == None or len(data)==0:
            return False
        for row1 in data:
            for row2 in data:
                if not(self.is_satisfy(row1,row2)):
                    return False
        return True
    # return true if self is a logical implication of DC
    # i.e self implies (logicaly) DC
    # uses Lemma 2
    def implies(self,DC):
        for i in range(len(self.predicates)):
            pred1 = self.predicates[i]
            pred2 = DC.predicates[i]
            if not(pred2.operator <= pred1.operator):
                return False
        return True
    
    def is_trivial(self):
        for pred in self.predicates:
            if len(pred.operator) != 3:
                return False
        return True
    
    #satisfiable : see definition section 2.3.1.1
    def satisfiable(self):
        for pred in self.predicates:
            if "=" not in pred.operator:
                return True
        return False
            
    def get_variations(self,theta):
        return self.get_var_recur(self,abs(theta),0)
      
    def get_var_recur(self,dc,theta,i):
        returnList = []
        if i < len(dc.predicates):
            if self.var_cost(dc) >= theta:
                return returnList
            else:
                weak_pred = self.predicates[i].get_weaker_not_empty()
                for elem in weak_pred:
                    new_dc = copy.deepcopy(dc)
                    new_dc.predicates[i] = elem
                    returnList = returnList + self.get_var_recur(new_dc,theta,i+1)
        else:
            returnList.append(dc)
        return returnList
        
    def is_weaker(self,DC):
        return DC.implies(self)
    
    #give the variation cost between self.DC and parameter DC
    def var_cost(self,DC,alpha = 1):
        sumCost = 0
        for i in range(self.size()):
            sumCost = sumCost + self.predicates[i].var_cost(DC.predicates[i],alpha)
        return sumCost
    
    def get_tuples_viol(self,viol):
        tupleViol = []
        for elem in viol:
            for tuple in elem:
                if not(tuple in tupleViol):
                    tupleViol.append(tuple)
        return tupleViol
    
    def get_cell(self, index1, index2):
        cell = []
        for pred in self.predicates:
            if not(len(pred.operator)==3):
                cell.append((pred.attribute,index1))
                cell.append((pred.attribute,index2))
        return cell
    
    def get_cell_viol(self,data):
        viol = self.get_violations(data)
        cell_viol = []
        for row1 in viol:
            for row2 in viol :
                cell = get_cell(row1,row2)
                for elem in cell:
                    if not(elem in cell_viol):
                        cell_viol.append(cell)
        return cell_viol
    
    def get_conflit_graph(self,dataSize,viol):
        #init
        compteur = []
        for i in range(dataSize):
            compteur.append(0)
        #on compte chaque fois qu'un elem apparait dans viol
        for elem in viol:
            for row in elem:
                compteur[row] = compteur[row]+1
        return compteur
        
    # return index of the maximum number of the list    
    def get_max(self,liste):
        max_index = 0
        max_elem = liste[0]
        for i in range(len(liste)):
            if liste[i] > max_index:
                max_elem = liste[i]
                max_index = i
        return max_index
    
    def get_vertex_cover(self,data):
        viol = self.get_violations(data)
        graphe = self.get_conflit_graph(len(data),viol)
        cover = []
        while(viol):
            indice = self.get_max(graphe)
            cover.append(indice)
            viol_deleted = []
            for elem in viol:
                if not(elem[0]==indice or elem[1]==indice):
                    viol_deleted.append(elem)
            viol = copy.deepcopy(viol_deleted)
            graphe = self.get_conflit_graph(len(data),viol)
        return cover
    
    def get_degree(self):
        return len(self.get_cell(1,0))
    
    def attribute_to_repair(self,weight):
        max_weight = 0#indice de l'attribut de poid max
        max_val = 9000 #supposons qu'aucun poid ne dépasse 9000
        for i in range(len(weight)):
            if(weight[i] < max_val)and(self.predicates[i].size() !=3):
                max_weight = i
                max_val = weight[i]
        return max_weight
    
    def is_suspect(self,row1,row2,att_to_repair):
        for i in range(len(self.predicates)):
            if (i != att_to_repair):
                if (self.predicates[i].size() !=3):
                    if not(self.predicates[i].is_true(row1,row2)):
                        return False
        return True
    
    def get_susp(self,cover,data,weight):
        suspect = []
        att_to_repair = self.attribute_to_repair(weight)
        for t in cover:
            for i in range(len(data)):
                if(t != i):
                    if(self.is_suspect(data[t],data[i],att_to_repair)):
                        tupleSuspect = (t,i)
                        suspect.append(tupleSuspect)
                    if(self.is_suspect(data[i],data[t],att_to_repair)):
                        tupleSuspect2 = (i,t)
                        suspect.append(tupleSuspect2)
        return suspect

In [215]:
class Database:
    
    def __init__(self,filename):
        
        self.colType = []
        self.colName = []
        self.sigma = []
        self.data = []
        self.weight = []
        self.readfile(filename)
        
    def readfile(self,filename):
        #Read file data.txt
        file = open(filename,"r")
        data = file.readlines()
        self.colName = data[0]
        del data[0]
        self.colType = data[0]
        self.colType = self.colType.split()
        del data[0]
        # now data is a list of list: each list in data is a row of the database
        for i in range (len(data)):
            data[i] =  data[i].split()
        self.data = copy.deepcopy(data)
        for i in range(len(self.colType)):
            self.weight.append(1)
        
            
    def set_weight(self,index,value):
        self.weight[index] = value
        
    def add_DC(self,dc):
        self.sigma.append(dc)
    
    def is_satisfy_by(self, Sigma):
        for dc in Sigma:
            if not(len(dc.get_violation)==0) :
                return False
        return True
    
    def is_satisfy(self):
        return is_satisfy_by(self.Sigma)
    
    def size(self):
        return len(self.data)
    
    def get_degree(self,sigma):
        degree = 0
        for elem in sigma:
            degree = degree + elem.get_degree()
        return degree
    
    def get_graphe(self):
        graphe = []
        for dc in self.sigma:
            graphe = graphe + dc.get_conflit_graph(self.size(),dc.get_violations(self.data))
        return set(graphe)
        
    def get_lower_bound(self,sigma):
        graphe = self.get_graphe()
        degree = self.get_degree(sigma)
        if(degree >0):
            return len(graphe)/degree
        else:
            return 0
    def get_upper_bound(self):
        dist_max = 1.5
        graphe = self.get_graphe()
        return len(graphe)*int(dist_max)
    
    def minval(self,index):
        typ = type_dict[self.colType[index]]
        mini = typ(self.data[0][index])
        for elem in self.data:
            if mini > typ(elem[index]):
                mini = typ(elem[index])
        return mini
    
    def maxval(self,index):
        typ = type_dict[self.colType[index]]
        maxi = typ(self.data[0][index])
        for elem in self.data:
            if maxi < typ(elem[index]):
                maxi = typ(elem[index])
        return maxi
    
    def repair_a_value(self,att_to_repair,row_to_repair,suspect,dc):
        epsilon = 0.00000001
        borne_min = self.minval(att_to_repair) -1
        borne_max = self.maxval(att_to_repair) +1
        op = dc.predicates[att_to_repair].get_inverse_op()
        typ = type_dict[self.colType[att_to_repair]]
        for elem in suspect:
            if abs(borne_max - borne_min) <= epsilon:
                return borne_max
            if(elem[0]==row_to_repair):
                if(set(op) ==set(["<","="])):
                    borne_max = typ(self.data[elem[1]][att_to_repair])
                if(set(op) ==set(["=",">"])):
                    borne_min = typ(self.data[elem[1]][att_to_repair])
                elif(set(op) ==set(["<"]) or set(op) ==set(["<",">"]) or set(op) ==set([">"])):
                    #borneMaxStricte
                    return None
                elif(set(op) == set(["="])):
                    return typ(self.data[elem[1]][att_to_repair])
            if(elem[1]==row_to_repair):
                if(set(op) ==set(["<","="])):
                    borne_min = typ(self.data[elem[0]][att_to_repair])
                if(set(op) ==set(["=",">"])):
                    borne_max = typ(self.data[elem[0]][att_to_repair])
                elif(set(op) ==set(["<"]) or set(op) ==set(["<",">"]) or set(op) ==set([">"])):
                    #borneStricte
                    return None
                elif(set(op) == set(["="])):
                    return typ(self.data[elem[0]][att_to_repair])     
        return None
    
    def data_repair(self,sigma,weight):
        new_rel = copy.deepcopy(self)
        new_rel.sigma = sigma
        for dc in sigma:
            cover = dc.get_vertex_cover(self.data)
            att_to_repair = dc.attribute_to_repair(weight)
            suspect = dc.get_susp(cover,self.data,weight)
            for elem in cover:
                repair = self.repair_a_value(att_to_repair,elem,suspect,dc)
                if repair != None:
                    new_rel.data[elem][att_to_repair] = repair
                else:
                    new_rel.data[elem][att_to_repair] = "?"
        return new_rel
    #Todo 
    #def write(filename):
    
    #def Theta-tolerant(thetha):
    def repair_cost(self,new_rel,weight):
        cost = 0
        data1 = self.data
        data2 = new_rel.data
        for i in range(len(data1)):
            for j in range(len(weight)):
                if(data[i][j] == "?"):
                    cost = cost + weight[j]*1.5
                elif(data1[i][j]!=data2[i][j]):
                    cost = cost + weight[j]*1
        return cost
    
    def theta_tolerant(self,variations,weight):
        return_relation = self.data_repair(self.sigma,weight)
        d_min = self.repair_cost(return_relation,weight)
        sigma = copy.deepcopy(self.sigma)
        for var in variations:
            if(self.get_lower_bound(var) <= d_min):
                new_rel = self.data_repair(var,weight)
                cost = self.repair_cost(new_rel,weight)
                if(cost <= d_min):
                    d_min = cost
                    return_relation = new_rel
                    sigma = var
        return_relation.sigma = sigma
        return return_relation

In [216]:
#Launche a Database
db = Database("Data.txt")
data = db.data

#create some Predicate
pred1 = Predicate(4,[">"],"int")
pred2 = Predicate(5,["<"],"int")
pred3 = Predicate(5,["<","="],"int")
#create a DC : the basic DC in our examples
den1 = DC(len(db.colType), db.colType)
den1.change_predicate(pred1)
den1.change_predicate(pred3)
sigma1 = [den1]
#create the DC that should be taken with theta-tolerant algorithm (refined)
den2 = DC(len(db.colType), db.colType)
den2.change_predicate(pred1)
den2.change_predicate(pred2)
sigma2 = [den2]

In [217]:
#look at the variations of den1 (with theta =1 because we don't want to over-refined)
#We should get the same DC as den2
var = den1.get_variations(1)
for elem in var:
    print(elem)
#Look at the violation with den1:
print(den1.get_violations(data))
#Look at the violation with den2:
print(den2.get_violations(data))

{({'attribute': 0, 'operator': ['<', '=', '>'], 'attributeType': 'string'}),({'attribute': 1, 'operator': ['<', '=', '>'], 'attributeType': 'string'}),({'attribute': 2, 'operator': ['<', '=', '>'], 'attributeType': 'string'}),({'attribute': 3, 'operator': ['<', '=', '>'], 'attributeType': 'int'}),({'attribute': 4, 'operator': ['>'], 'attributeType': 'int'}),({'attribute': 5, 'operator': ['<'], 'attributeType': 'int'}),}
{({'attribute': 0, 'operator': ['<', '=', '>'], 'attributeType': 'string'}),({'attribute': 1, 'operator': ['<', '=', '>'], 'attributeType': 'string'}),({'attribute': 2, 'operator': ['<', '=', '>'], 'attributeType': 'string'}),({'attribute': 3, 'operator': ['<', '=', '>'], 'attributeType': 'int'}),({'attribute': 4, 'operator': ['>'], 'attributeType': 'int'}),({'attribute': 5, 'operator': ['='], 'attributeType': 'int'}),}
{({'attribute': 0, 'operator': ['<', '=', '>'], 'attributeType': 'string'}),({'attribute': 1, 'operator': ['<', '=', '>'], 'attributeType': 'string'}),(

In [218]:
#look at the cover:
cover1 = den1.get_vertex_cover(data)
print(cover1)
cover2 = den2.get_vertex_cover(data)
print(cover2)
#We can clearly expect (but there's no sufficient proof here) that
#den2 will be better for repairing

[6, 4, 5, 1, 2]
[3]


In [219]:
#look at the suspect now
suspect = den1.get_susp(cover1,data,[99,99,99,99,5,1])
print(suspect)
suspect2 = den2.get_susp(cover2,data,[99,99,99,99,5,1])
print(suspect2)

[(6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (7, 6), (8, 6), (9, 6), (4, 0), (4, 1), (4, 2), (4, 3), (6, 4), (7, 4), (8, 4), (9, 4), (5, 0), (5, 1), (5, 2), (5, 3), (6, 5), (7, 5), (8, 5), (9, 5), (1, 0), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (2, 0), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 2), (9, 2)]
[(3, 0), (3, 1), (3, 2), (4, 3), (5, 3), (6, 3), (7, 3), (8, 3), (9, 3)]


In [220]:
weight = [99,99,99,99,5,1] #this the w(A) in the repair cost formula
#This is the repair if we don't change any DC
repair = db.data_repair(sigma1,weight)
for row in repair.data:
    print(row)
#We got 5 fresh variable ("?" in this implementation)
db.repair_cost(repair,weight)

['Ayres', '8-8-1984', '322-573', '2007', '21', '0']
['Ayres', '5-1-1960', '***-389', '2007', '22', '?']
['Ayres', '5-1-1960', '564-389', '2007', '22', '?']
['Stanley', '13-8-1987', '868-701', '2007', '23', '3']
['Stanley', '31-7-1983', '***-198', '2007', '24', '?']
['Stanley', '31-7-1983', '930-198', '2008', '24', '?']
['Dustin', '2-12-1985', '179-924', '2008', '25', '?']
['Dustin', '5-9-1980', '***-870', '2008', '100', '21']
['Dustin', '5-9-1980', '824-870', '2009', '100', '21']
['Dustin', '9-4-1984', '387-215', '2009', '150', '40']


5

In [221]:
#This is the repair if we don't change any DC
repair2 = db.data_repair(sigma2,weight)
for row in repair2.data:
    print(row)
#We got 5 fresh variable ("?" in this implementation)
db.repair_cost(repair2,weight)

['Ayres', '8-8-1984', '322-573', '2007', '21', '0']
['Ayres', '5-1-1960', '***-389', '2007', '22', '0']
['Ayres', '5-1-1960', '564-389', '2007', '22', '0']
['Stanley', '13-8-1987', '868-701', '2007', '23', 0]
['Stanley', '31-7-1983', '***-198', '2007', '24', '0']
['Stanley', '31-7-1983', '930-198', '2008', '24', '0']
['Dustin', '2-12-1985', '179-924', '2008', '25', '0']
['Dustin', '5-9-1980', '***-870', '2008', '100', '21']
['Dustin', '5-9-1980', '824-870', '2009', '100', '21']
['Dustin', '9-4-1984', '387-215', '2009', '150', '40']


1

In [222]:
var2 = []
db.sigma = sigma1
for elem in var:
    var2.append([elem])
thetarepair = db.theta_tolerant(var2,[99,99,99,99,5,1])
for row in thetarepair.data:
    print(row)

['Ayres', '8-8-1984', '322-573', '2007', '21', '0']
['Ayres', '5-1-1960', '***-389', '2007', '22', '0']
['Ayres', '5-1-1960', '564-389', '2007', '22', '0']
['Stanley', '13-8-1987', '868-701', '2007', '23', 0]
['Stanley', '31-7-1983', '***-198', '2007', '24', '0']
['Stanley', '31-7-1983', '930-198', '2008', '24', '0']
['Dustin', '2-12-1985', '179-924', '2008', '25', '0']
['Dustin', '5-9-1980', '***-870', '2008', '100', '21']
['Dustin', '5-9-1980', '824-870', '2009', '100', '21']
['Dustin', '9-4-1984', '387-215', '2009', '150', '40']


In [181]:
#Launch a Database
db = Database("Data.txt")
data = db.data

#Create some DC


den1 = DC(len(db.colType), db.colType)
den1.change_predicate(pred1)
print(den1.is_satisfy(data[0],data[1]) == True) 
print(den1.is_satisfy(data[1],data[0]) == False) 
print(den1.size()==6)
den1.change_predicate(pred2)
print(den1.is_satisfy(data[0],data[1])==True) 
print(den1.is_satisfy(data[1],data[0])==True)
print(den1.size()==6)
print(den1.is_satisfy(data[4],data[3])==False) 

print(den1.get_violations(data) == [(4, 3), (5, 3), (6, 3)])

True
True
True
True
True
True
True
True


In [182]:
#test trivialité

den2 = DC(len(db.colType), db.colType)
print(den2.is_trivial()==True)
#test implies
pred3 = Predicate(5,["<",">"],db.colType[5])
den2.change_predicate(pred1)
den2.change_predicate(pred3)
print(den2.is_satisfy_by_data(data)==False)
print(den1.implies(den2) ==False)
print(den2.implies(den1) ==True)
print(den2.is_trivial()==False)
#test satisfiable
print(den2.satisfiable())

True
True
True
True
True
True


In [183]:
var = den1.get_variations(1)
for elem in var:
    print(elem)
print(len(var))

{({'attribute': 0, 'operator': ['<', '=', '>'], 'attributeType': 'string'}),({'attribute': 1, 'operator': ['<', '=', '>'], 'attributeType': 'string'}),({'attribute': 2, 'operator': ['<', '=', '>'], 'attributeType': 'string'}),({'attribute': 3, 'operator': ['<', '=', '>'], 'attributeType': 'int'}),({'attribute': 4, 'operator': ['>'], 'attributeType': 'int'}),({'attribute': 5, 'operator': ['<'], 'attributeType': 'int'}),}
1


In [184]:
#vertex cover
u = den1.get_tuples_viol(den1.get_violations(data))
viol = den1.get_violations(data)
print(len(u)==4)
print(u)
print(viol == [(4, 3), (5, 3), (6, 3)])
den1.get_cell(1,0)
cover = den1.get_vertex_cover(data)
print(cover)
#need more test, ici c'est un cas particulier

True
[4, 3, 5, 6]
True
[3]


In [185]:
suspect = den1.get_susp(cover,data,[99,99,99,99,5,1])
print(suspect)

[(3, 0), (3, 1), (3, 2), (4, 3), (5, 3), (6, 3), (7, 3), (8, 3), (9, 3)]


In [687]:
den1.is_suspect(data[3],data[1],5)

True

In [688]:
att_to_repair = den1.attribute_to_repair([99,99,99,99,5,1])
print(att_to_repair)

5


In [689]:
row_to_repair =3
k = db.repair_a_value(att_to_repair,row_to_repair,suspect,den1)
print(k)

0


In [690]:
sigma1 = [den1]
repair = db.data_repair(sigma1,[99,99,99,99,5,1])
for elem in repair.data:
    print(elem)

['Ayres', '8-8-1984', '322-573', '2007', '21', '0']
['Ayres', '5-1-1960', '***-389', '2007', '22', '0']
['Ayres', '5-1-1960', '564-389', '2007', '22', '0']
['Stanley', '13-8-1987', '868-701', '2007', '23', 0]
['Stanley', '31-7-1983', '***-198', '2007', '24', '0']
['Stanley', '31-7-1983', '930-198', '2008', '24', '0']
['Dustin', '2-12-1985', '179-924', '2008', '25', '0']
['Dustin', '5-9-1980', '***-870', '2008', '100', '21']
['Dustin', '5-9-1980', '824-870', '2009', '100', '21']
['Dustin', '9-4-1984', '387-215', '2009', '150', '40']


In [691]:
db.repair_cost(repair,[99,99,99,99,5,1])

1

In [692]:
den4 = DC(len(db.colType), db.colType)
pred3 = Predicate(5,["<","="],"int")
den4.change_predicate(pred1)
den4.change_predicate(pred3)
db.sigma = [den4]
var = den4.get_variations(1)
for elem in var:
    print(elem)
print(len(var))

{({'attribute': 0, 'operator': ['<', '=', '>'], 'attributeType': 'string'}),({'attribute': 1, 'operator': ['<', '=', '>'], 'attributeType': 'string'}),({'attribute': 2, 'operator': ['<', '=', '>'], 'attributeType': 'string'}),({'attribute': 3, 'operator': ['<', '=', '>'], 'attributeType': 'int'}),({'attribute': 4, 'operator': ['>'], 'attributeType': 'int'}),({'attribute': 5, 'operator': ['<'], 'attributeType': 'int'}),}
{({'attribute': 0, 'operator': ['<', '=', '>'], 'attributeType': 'string'}),({'attribute': 1, 'operator': ['<', '=', '>'], 'attributeType': 'string'}),({'attribute': 2, 'operator': ['<', '=', '>'], 'attributeType': 'string'}),({'attribute': 3, 'operator': ['<', '=', '>'], 'attributeType': 'int'}),({'attribute': 4, 'operator': ['>'], 'attributeType': 'int'}),({'attribute': 5, 'operator': ['='], 'attributeType': 'int'}),}
{({'attribute': 0, 'operator': ['<', '=', '>'], 'attributeType': 'string'}),({'attribute': 1, 'operator': ['<', '=', '>'], 'attributeType': 'string'}),(

In [148]:
var2 = []
for elem in var:
    poop = [elem]
    var2.append(poop)
kk = db.theta_tolerant(var2,[99,99,99,99,5,1])
print(kk.data)
for elem in kk.sigma:
    print(elem)

UnboundLocalError: local variable 'returnRelation' referenced before assignment

In [694]:
rep=kk.data_repair(kk.sigma,[99,99,99,99,5,1])
for elem in rep.data:
    print(elem)
for elem in rep.sigma:
    for pred in elem.predicates:
        print(pred)

['Ayres', '8-8-1984', '322-573', '2007', '21', '0']
['Ayres', '5-1-1960', '***-389', '2007', '22', '0']
['Ayres', '5-1-1960', '564-389', '2007', '22', '0']
['Stanley', '13-8-1987', '868-701', '2007', '23', 0]
['Stanley', '31-7-1983', '***-198', '2007', '24', '0']
['Stanley', '31-7-1983', '930-198', '2008', '24', '0']
['Dustin', '2-12-1985', '179-924', '2008', '25', '0']
['Dustin', '5-9-1980', '***-870', '2008', '100', '21']
['Dustin', '5-9-1980', '824-870', '2009', '100', '21']
['Dustin', '9-4-1984', '387-215', '2009', '150', '40']
({'attribute': 0, 'operator': ['<', '=', '>'], 'attributeType': 'string'})
({'attribute': 1, 'operator': ['<', '=', '>'], 'attributeType': 'string'})
({'attribute': 2, 'operator': ['<', '=', '>'], 'attributeType': 'string'})
({'attribute': 3, 'operator': ['<', '=', '>'], 'attributeType': 'int'})
({'attribute': 4, 'operator': ['>'], 'attributeType': 'int'})
({'attribute': 5, 'operator': ['<'], 'attributeType': 'int'})


In [12]:
dc = copy.deepcopy(den1)
predx = Predicate(1,[">","="],"str")
dc.change_predicate(predx)
den1.var_cost(dc)

1